In [1]:
import pandas as pd

In [2]:
# Connect to database

from sqlalchemy import create_engine

URI="localhost"
PORT="5433"
DB = "jetaDb"
USER = "postgres"
PASSWORD = "00001234"
  
engine = create_engine("postgresql://{}:{}@{}:{}/{}".format(USER, PASSWORD, URI, PORT, DB), echo=True)

/home/isaac/anaconda3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [97]:
# Variables from query

start = "767" # ucd
destination = "1914" # old cabra road
lineid = "39A"
start_time = "1530008000"

# http://localhost:8000/main/journeytime?source=767&destination=1914&lineid=39A&time=1530008000

# Rain should be gotten from api or else table

rain = 0.5

In [167]:
# Transform time into variables required from model

import time
from datetime import datetime, timedelta
from pytz import timezone

# Get Irish timezone (utc + daylight saving time (DST))
irish_time = timezone('Europe/Dublin')
print("Irish Standard Time: ", irish_time)

# Get unixtime as datetime object
dt_time = datetime.fromtimestamp(int(start_time), irish_time)
print("Datetime: ", dt_time)

# Get day of week -> Mon: 0, Sun: 6
weekday = dt_time.weekday()
print("Weekday: ", weekday)

# Create list with desired weekday filled.
week_dummies = [0] * 7
week_dummies[weekday] = 1
del week_dummies[2] # Delete wednesday - not included in model due to dummy var trap
print("Week dummies: ", week_dummies)

# Get arrivaltime in seconds
date = dt_time.date()
date_unixtime = time.mktime(date.timetuple())
seconds_since_midnight = int(time.mktime((dt_time - timedelta(seconds = date_unixtime)).timetuple()))
print("Seconds since midnight (arrival time): ", seconds_since_midnight)


Irish Standard Time:  Europe/Dublin
Datetime:  2018-06-26 11:13:20+01:00
Weekday:  1
Week dummies:  [0, 1, 0, 0, 0, 0]
Seconds since midnight (arrival time):  40400


In [169]:
# Model inputs

model_inputs = [seconds_since_midnight, rain] + week_dummies
model_inputs

[40400, 0.5, 0, 1, 0, 0, 0, 0]

In [38]:
# Get stop lists associated with this lineid, start stop and end stop

sql = """

SELECT * 
FROM main_routes 
WHERE routeid IN (
    SELECT UNNEST(routes) 
    FROM main_lines 
    WHERE main_lines.lineid = '{0}'
) 
AND '{1}' = ANY(main_routes.stopids) 
AND '{2}' = ANY(main_routes.stopids) 
;

""".format(lineid, start, destination)

routes = pd.read_sql(sql, engine)

# Lines.objects.all
# Routes.objects.all(routeid )

2018-07-08 21:16:15,188 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2018-07-08 21:16:15,190 INFO sqlalchemy.engine.base.Engine {'name': "\n\nSELECT * \nFROM main_routes \nWHERE routeid IN (\n    SELECT UNNEST(routes) \n    FROM main_lines \n    WHERE main_lines.lineid = '39A'\n) \nAND '767' = ANY(main_routes.stopids) \nAND '1914' = ANY(main_routes.stopids) \n;\n\n"}
2018-07-08 21:16:15,382 INFO sqlalchemy.engine.base.Engine 

SELECT * 
FROM main_routes 
WHERE routeid IN (
    SELECT UNNEST(routes) 
    FROM main_lines 
    WHERE main_lines.lineid = '39A'
) 
AND '767' = ANY(main_routes.stopids) 
AND '1914' = ANY(main_routes.stopids) 
;


2018-07-08 21:16:15,384 INFO sqlalchemy.engine.base.Engine {}


In [39]:
routes

,routeid,direction,stopids
0,39A_40,1,"[767, 768, 769, 770, 771, 772, 773, 774, 775, ..."


In [149]:
if routes.shape[0] > 1:
    print("Error: multiple possible routes.")
    print(routes)

In [43]:
# Convert list of stopids to list

stop_list = routes['stopids'].tolist()[0]
print(stop_list)

[767, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 779, 780, 781, 782, 783, 784, 785, 786, 793, 7576, 7586, 7587, 7588, 328, 1443, 1444, 1445, 1647, 1648, 1649, 1911, 1913, 1914, 1805, 1806, 1660, 1661, 1662, 1664, 1665, 1666, 1807, 7167, 1808, 7389, 7025, 4464, 1869, 1870, 1871, 1872, 1873, 1874, 1875, 1876, 1877, 1878, 1879, 1899, 6107, 6108, 6109, 6110, 7020, 7029, 7038, 7011, 2171, 7160, 7047, 7161, 7162]


In [48]:
# Slice list by start and destination stop

journey_stops = stop_list[stop_list.index(int(start)):(stop_list.index(int(destination))+1)]
print(journey_stops)

[767, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 779, 780, 781, 782, 783, 784, 785, 786, 793, 7576, 7586, 7587, 7588, 328, 1443, 1444, 1445, 1647, 1648, 1649, 1911, 1913, 1914]


In [50]:
# Change each stopid into string
stringified = list(map(str, journey_stops))

# Zip ['0', .. 'n'] and ['1', .., 'n'] into list [('0', '1'), .., ('n - 1', 'n')]
# Join tuples to make ['1_2', .., 'n-1_n']
journey_segments = [ '_'.join(x) for x in zip(stringified[0:], stringified[1:])]
print(journey_segments)

['767_768', '768_769', '769_770', '770_771', '771_772', '772_773', '773_774', '774_775', '775_776', '776_777', '777_779', '779_780', '780_781', '781_782', '782_783', '783_784', '784_785', '785_786', '786_793', '793_7576', '7576_7586', '7586_7587', '7587_7588', '7588_328', '328_1443', '1443_1444', '1444_1445', '1445_1647', '1647_1648', '1648_1649', '1649_1911', '1911_1913', '1913_1914']


In [172]:
# Select coefficient rows with these segment ids

sql2 = """

SELECT *
FROM main_coefficients
WHERE segment = ANY(ARRAY{0})

""".format(journey_segments)

coefficients = pd.read_sql(sql2, engine)

2018-07-09 00:56:22,034 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2018-07-09 00:56:22,036 INFO sqlalchemy.engine.base.Engine {'name': "\n\nSELECT *\nFROM main_coefficients\nWHERE segment = ANY(ARRAY['767_768', '768_769', '769_770', '770_771', '771_772', '772_773', '773_774', '774_775 ... (158 characters truncated) ... 7', '7587_7588', '7588_328', '328_1443', '1443_1444', '1444_1445', '1445_1647', '1647_1648', '1648_1649', '1649_1911', '1911_1913', '1913_1914'])\n\n"}
2018-07-09 00:56:22,801 INFO sqlalchemy.engine.base.Engine 

SELECT *
FROM main_coefficients
WHERE segment = ANY(ARRAY['767_768', '768_769', '769_770', '770_771', '771_772', '772_773', '773_774', '774_775', '775_776', '776_777', '777_779', '779_780', '780_781', '781_782', '782_783', '783_784', '784_785', '785_786', '786_793', '793_7576', '7576_7586', '7586_7587', '7587_7588', '7588_328', '328

In [173]:
coefficients.head(5)

,segment,intercept,arrivaltime,rain,dayofweek_Friday,dayofweek_Monday,dayofweek_Saturday,dayofweek_Sunday,dayofweek_Thursday,dayofweek_Tuesday
0,328_1443,132.621983,0.000372,-1.012906,1.844373,-1.666448,-5.032170,-3.358479,-0.482329,-0.020099
1,783_784,40.640856,0.000006,0.866560,-0.809178,-2.476708,-8.547565,-10.007176,-0.638526,-0.218791
2,7586_7587,122.372920,0.000147,0.237121,0.438237,-4.404150,-16.039802,-18.645662,-1.922852,0.211988
3,775_776,78.305410,-0.000352,0.489834,0.179928,-4.515035,-3.676717,-7.138404,-3.106753,-1.760917
4,779_780,45.017656,-0.000158,0.639585,-0.397866,-0.665312,-4.545622,-5.479638,-0.478755,-0.075373


In [174]:
# Sort values by journey_segment segmentid

coefficients['segment'] = coefficients['segment'].astype("category")
coefficients['segment'].cat.set_categories(journey_segments, inplace=True)
coefficients = coefficients.sort_values(["segment"])
coefficients.head(5)

,segment,intercept,arrivaltime,rain,dayofweek_Friday,dayofweek_Monday,dayofweek_Saturday,dayofweek_Sunday,dayofweek_Thursday,dayofweek_Tuesday
11,767_768,123.285491,-0.000179,1.266745,-2.018558,0.834109,-10.095867,-10.386478,-1.594662,-2.440293
22,768_769,70.940947,0.000006,0.853329,-0.247459,-4.777141,-12.717419,-19.586873,-3.682586,-2.967836
16,769_770,70.868285,-0.000202,0.290486,-1.482873,-2.142149,-10.079057,-11.424488,-2.165400,-1.492595
25,770_771,34.436680,-0.000044,-0.081414,-0.369489,-1.599961,-2.655441,-3.580868,-0.951509,-0.013504
30,771_772,60.742119,-0.000327,1.154692,0.668076,-0.813195,-7.692121,-6.731297,0.148827,-0.540391


In [175]:
# Rearrange columns and set segment id as index

coefficients = coefficients[["segment", "intercept", "arrivaltime", "rain", "dayofweek_Monday", "dayofweek_Tuesday", 
                        "dayofweek_Thursday", "dayofweek_Friday", "dayofweek_Saturday", "dayofweek_Sunday"]]
coefficients = coefficients.set_index('segment')
coefficients.head(5)

,intercept,arrivaltime,rain,dayofweek_Monday,dayofweek_Tuesday,dayofweek_Thursday,dayofweek_Friday,dayofweek_Saturday,dayofweek_Sunday
segment,,,,,,,,,
767_768,123.285491,-0.000179,1.266745,0.834109,-2.440293,-1.594662,-2.018558,-10.095867,-10.386478
768_769,70.940947,0.000006,0.853329,-4.777141,-2.967836,-3.682586,-0.247459,-12.717419,-19.586873
769_770,70.868285,-0.000202,0.290486,-2.142149,-1.492595,-2.165400,-1.482873,-10.079057,-11.424488
770_771,34.436680,-0.000044,-0.081414,-1.599961,-0.013504,-0.951509,-0.369489,-2.655441,-3.580868
771_772,60.742119,-0.000327,1.154692,-0.813195,-0.540391,0.148827,0.668076,-7.692121,-6.731297


In [178]:
arrivaltime = model_inputs[0]
totaltraveltime = 0
segment_times = []

for i, rows in coefficients.iterrows():
    traveltime = (rows['intercept']
                  +(rows['arrivaltime']*arrivaltime)
                  +(rows['rain']*model_inputs[1])
                  +(rows['dayofweek_Friday']*model_inputs[2])
                  +(rows['dayofweek_Monday']*model_inputs[3])
                  +(rows['dayofweek_Saturday']*model_inputs[4])
                  +(rows['dayofweek_Sunday']*model_inputs[5])
                  +(rows['dayofweek_Thursday']*model_inputs[6])
                  +(rows['dayofweek_Tuesday']*model_inputs[7]))
    
    segment_times.append((i, round(traveltime)))
    totaltraveltime += traveltime
    arrivaltime = model_inputs[0] + totaltraveltime # initial start time + sum of previous segment times
        
print('Arrival Time:', int(arrivaltime))
print('Total Travel Time:', int(totaltraveltime))
#print('Segment Times:', segment_times)
segment_times

Arrival Time: 42907
Total Travel Time: 2507


[('767_768', 118.0),
 ('768_769', 67.0),
 ('769_770', 61.0),
 ('770_771', 31.0),
 ('771_772', 47.0),
 ('772_773', 56.0),
 ('773_774', 63.0),
 ('774_775', 59.0),
 ('775_776', 60.0),
 ('776_777', 45.0),
 ('777_779', 74.0),
 ('779_780', 38.0),
 ('780_781', 36.0),
 ('781_782', 67.0),
 ('782_783', 81.0),
 ('783_784', 39.0),
 ('784_785', 65.0),
 ('785_786', 102.0),
 ('786_793', 154.0),
 ('793_7576', 43.0),
 ('7576_7586', 98.0),
 ('7586_7587', 124.0),
 ('7587_7588', 37.0),
 ('7588_328', 157.0),
 ('328_1443', 146.0),
 ('1443_1444', 66.0),
 ('1444_1445', 79.0),
 ('1445_1647', 133.0),
 ('1647_1648', 96.0),
 ('1648_1649', 52.0),
 ('1649_1911', 89.0),
 ('1911_1913', 96.0),
 ('1913_1914', 30.0)]

In [179]:
total = 0
for i in segment_times:
    total += i[1]
    
print(total)

2509.0


In [180]:
# Construct json

json_dict = {}
json_dict['arrivaltime'] = round(arrivaltime)
json_dict['totaltraveltime'] = round(totaltraveltime)
json_dict['segment_times'] = {}

for i in segment_times:
    json_dict['segment_times'][i[0]] = i[1]

In [181]:
json_dict 

{'arrivaltime': 42907.0,
 'segment_times': {'1443_1444': 66.0,
  '1444_1445': 79.0,
  '1445_1647': 133.0,
  '1647_1648': 96.0,
  '1648_1649': 52.0,
  '1649_1911': 89.0,
  '1911_1913': 96.0,
  '1913_1914': 30.0,
  '328_1443': 146.0,
  '7576_7586': 98.0,
  '7586_7587': 124.0,
  '7587_7588': 37.0,
  '7588_328': 157.0,
  '767_768': 118.0,
  '768_769': 67.0,
  '769_770': 61.0,
  '770_771': 31.0,
  '771_772': 47.0,
  '772_773': 56.0,
  '773_774': 63.0,
  '774_775': 59.0,
  '775_776': 60.0,
  '776_777': 45.0,
  '777_779': 74.0,
  '779_780': 38.0,
  '780_781': 36.0,
  '781_782': 67.0,
  '782_783': 81.0,
  '783_784': 39.0,
  '784_785': 65.0,
  '785_786': 102.0,
  '786_793': 154.0,
  '793_7576': 43.0},
 'totaltraveltime': 2507.0}